In [1]:
import pandas as pd
import numpy as np
import math
import pickle

from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
from scipy.io import loadmat

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,roc_auc_score,recall_score,precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from pyearth import Earth
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
from sklearn.model_selection import StratifiedKFold

from src import SMOTE
from src import CFS
from src import metrices_V2 as metrices

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path

# import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
/Users/suvodeepmajumder/Conda/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
def apply_smote(df):
    df.reset_index(drop=True,inplace=True)
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def apply_cfs(df):
        y = df.Bugs.values
        X = df.drop(labels = ['Bugs'],axis = 1)
        X = X.values
        selected_cols = CFS.cfs(X,y)
        cols = df.columns[[selected_cols]].tolist()
        cols.append('Bugs')
        return df[cols],cols

In [18]:
def load_both_data(project,metric):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    cols = understand_df.columns.tolist()
    understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    understand_df['Name'] = understand_df.Name.str.rsplit('.',1).str[1]
    
    commit_guru_file_level_path = 'data/commit_guru_file/' + project + '.csv'
    commit_guru_file_level_df = pd.read_csv(commit_guru_file_level_path)
    commit_guru_file_level_df['commit_hash'] = commit_guru_file_level_df.commit_hash.str.strip('"')
    commit_guru_file_level_df = commit_guru_file_level_df[commit_guru_file_level_df['file_name'].str.contains('.java')]
    commit_guru_file_level_df['Name'] = commit_guru_file_level_df.file_name.str.rsplit('/',1).str[1].str.split('.').str[0].str.replace('/','.')
    commit_guru_file_level_df = commit_guru_file_level_df.drop('file_name',axis = 1)
#     print(commit_guru_file_level_df.columns)
    
    df = understand_df.merge(commit_guru_file_level_df,how='left',on=['commit_hash','Name'])
    
    
    cols = df.columns.tolist()
    cols.remove('Bugs')
    cols.append('Bugs')
    df = df[cols]
    
    for item in ['Kind', 'Name','commit_hash']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
#     df.dropna(inplace=True)
    df = df.drop_duplicates()
    df.reset_index(drop=True, inplace=True)
    
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    imp_mean = IterativeImputer(random_state=0)
    X = imp_mean.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    
    if metric == 'process':
        X = X[['file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
       'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
       'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
       'file_ns', 'file_exp', 'file_sexp', 'file_rexp', 'file_nd', 'file_sctr']]
    elif metric == 'product':
        X = X.drop(['la', 'ld', 'lt', 'age', 'ndev', 'nuc', 'ns', 'exp', 'sexp', 'rexp', 'nd'],axis = 1)
    else:
        X = X
    return X,y


def load_data_commit_level(project,metric):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    cols = understand_df.columns.tolist()
    understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    understand_df['Name'] = understand_df.Name.str.rsplit('.',1).str[1]
    
    commit_guru_file_level_path = 'data/commit_guru_file_level/' + project + '_file.csv'
    commit_guru_file_level_df = pd.read_csv(commit_guru_file_level_path)
    commit_guru_file_level_df['commit_hash'] = commit_guru_file_level_df.commit_hash.str.strip('"')
    commit_guru_file_level_df = commit_guru_file_level_df[commit_guru_file_level_df['file_name'].str.contains('.java')]
    commit_guru_file_level_df['Name'] = commit_guru_file_level_df.file_name.str.rsplit('/',1).str[1].str.split('.').str[0].str.replace('/','.')
    commit_guru_file_level_df = commit_guru_file_level_df.drop('file_name',axis = 1)
    
    
    df = understand_df.merge(commit_guru_file_level_df,how='left',on=['commit_hash','Name'])
    
    
    cols = df.columns.tolist()
    cols.remove('Bugs')
    cols.append('Bugs')
    df = df[cols]
    commit_hash = df.commit_hash
    for item in ['Kind', 'Name','commit_hash']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
#     df.dropna(inplace=True)
    df = df.drop_duplicates()
    df.reset_index(drop=True, inplace=True)
    
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    imp_mean = IterativeImputer(random_state=0)
    X = imp_mean.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    
    if metric == 'process':
        X = X[['file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
       'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
       'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
       'file_ns', 'file_exp', 'file_sexp', 'file_rexp', 'file_nd', 'file_sctr']]
    elif metric == 'product':
        X = X.drop(['la', 'ld', 'lt', 'age', 'ndev', 'nuc', 'ns', 'exp', 'sexp', 'rexp', 'nd'],axis = 1)
    else:
        X = X
    
    df = X
    df['Bugs'] = y
    df['commit_hash'] = commit_hash
    unique_commits = df.commit_hash.unique()
    
    train_df = df[df.commit_hash.isin(unique_commits[0:int(len(unique_commits)*.6)])]
    test_df = df[df.commit_hash.isin(unique_commits[int(len(unique_commits)*.6):])]
    
    train_df.reset_index(inplace=True, drop=True)
    test_df.reset_index(inplace=True, drop=True)
    train_y = train_df.Bugs
    train_X = train_df.drop(['Bugs','commit_hash'],axis = 1)
    
    return train_X,train_y,test_df


In [16]:
def run_self_release(project,metric):
    precision = []
    recall = []
    pf = []
    f1 = []
    g_score = []
    auc = []
    pci_20 = []
    ifa = []
    X_train,y_train,test_df = load_data_commit_level(project,metric)      
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf =  RandomForestClassifier()
    clf.fit(X_train,y_train)
    importance = clf.feature_importances_
    unique_commits_list = np.array_split(test_df.commit_hash.unique(), 5)
    for i in range(len(unique_commits_list)):
        test_df_subset = test_df[test_df.commit_hash.isin(unique_commits_list[i])]
        y_test = test_df_subset.Bugs
        X_test = test_df_subset.drop(['Bugs','commit_hash'],axis = 1)
        if metric == 'process':
            loc = X_test['la'] + X_test['lt']
        elif metric == 'product':
            loc = X_test.CountLineCode
        else:
            loc = X_test['la'] + X_test['lt']                 
        predicted = clf.predict(X_test)
        abcd = metrices.measures(y_test,predicted,loc)
        pf.append(abcd.get_pf())
        recall.append(abcd.calculate_recall())
        precision.append(abcd.calculate_precision())
        f1.append(abcd.calculate_f1_score())
        g_score.append(abcd.get_g_score())
        pci_20.append(abcd.get_pci_20())
        ifa.append(abcd.get_ifa())
        try:
            auc.append(roc_auc_score(y_test, predicted))
        except:
            auc.append(0)
        print(classification_report(y_test, predicted))
#         print(recall,precision,pf,f1,g_score,auc,pci_20)
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance

In [29]:
def run_self_k(project,metric):
    precision = []
    recall = []
    pf = []
    f1 = []
    g_score = []
    auc = []
    pci_20 = []
    ifa = []
    importance = []
    X,y = load_both_data(project,metric)
    for _ in range(5):
        skf = StratifiedKFold(n_splits=5)
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X.loc[train_index], X.loc[test_index]
            y_train, y_test = y.loc[train_index], y.loc[test_index]
            if metric == 'process':
                loc = X_test['la'] + X_test['lt']
            elif metric == 'product':
                loc = X_test.CountLineCode
            else:
                loc = X_test['la'] + X_test['lt']
            df_smote = pd.concat([X_train,y_train],axis = 1)
            df_smote = apply_smote(df_smote)
            y_train = df_smote.Bugs
            X_train = df_smote.drop('Bugs',axis = 1)
            clf = RandomForestClassifier()
            clf.fit(X_train,y_train)
            importance = clf.feature_importances_
            predicted = clf.predict(X_test)
            abcd = metrices.measures(y_test,predicted,loc)
            pf.append(abcd.get_pf())
            recall.append(abcd.calculate_recall())
            precision.append(abcd.calculate_precision())
            f1.append(abcd.calculate_f1_score())
            g_score.append(abcd.get_g_score())
            pci_20.append(abcd.get_pci_20())
            ifa.append(abcd.get_ifa())
            try:
                auc.append(roc_auc_score(y_test, predicted))
            except:
                auc.append(0)
#             print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance

In [8]:
def run_self(project,metric):
    X,y = load_both_data(project,metric)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    if metric == 'process':
        loc = X_test['file_la'] + X_test['file_lt']
    elif metric == 'product':
        loc = X_test.CountLineCode
    else:
        loc = X_test['la'] + X_test['lt']
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf = RandomForestClassifier()
    clf.fit(X_train,y_train)
    importance = clf.feature_importances_
    print(len(importance))
    predicted = clf.predict(X_test)
    abcd = metrices.measures(y_test,predicted,loc)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    pci_20 = abcd.get_pci_20()
    ifa = abcd.get_ifa()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance

In [50]:
proj_df = pd.read_csv('projects.csv')
projects = proj_df.repo_name.tolist()

In [ ]:
precision_list = {}
recall_list = {}
pf_list = {}
f1_list = {}
g_list = {}
auc_list = {}
pci_20_list = {}
ifa_list = {}
featue_importance = {}
errored = []
for project in projects[150:]:
    try:
        if project == '.DS_Store':
            continue
#         if project != 'Discord4J':
#             continue
        print("+++++++++++++++++   "  + project + "  +++++++++++++++++")
        recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance = run_self(project,'process')
        recall_list[project] = recall
        precision_list[project] = precision
        pf_list[project] = pf
        f1_list[project] = f1
        g_list[project] = g_score
        auc_list[project] = auc
        pci_20_list[project] = pci_20
        ifa_list[project] = ifa
        featue_importance[project] = importance
    except Exception as e:
        errored.append(project)
        print(e)
        continue
final_result = {}
final_result['precision'] = precision_list
final_result['recall'] = recall_list
final_result['pf'] = pf_list
final_result['f1'] = f1_list
final_result['g'] = g_list
final_result['auc'] = auc_list
final_result['pci_20'] = pci_20_list
final_result['ifa'] = ifa_list
final_result['featue_importance'] = featue_importance

+++++++++++++++++   spring-xd-samples  +++++++++++++++++
Index(['commit_hash', 'file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
       'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
       'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
       'file_ns', 'file_exp', 'file_sexp', 'file_rexp', 'file_nd', 'file_sctr',
       'Name'],
      dtype='object')
Shape of passed values is (62, 45), indices imply (62, 66)
+++++++++++++++++   environment-dashboard  +++++++++++++++++
Index(['commit_hash', 'file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
       'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
       'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
       'file_ns', 'file_exp', 'file_sexp', 'file_rexp', 'file_nd', 'file_sctr',
       'Name'],
      dtype='object')
21
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         9
           1   

21
              precision    recall  f1-score   support

           0       1.00      0.96      0.98        94
           1       0.83      1.00      0.90        19

    accuracy                           0.96       113
   macro avg       0.91      0.98      0.94       113
weighted avg       0.97      0.96      0.97       113

+++++++++++++++++   PatternLock  +++++++++++++++++
Index(['commit_hash', 'file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
       'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
       'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
       'file_ns', 'file_exp', 'file_sexp', 'file_rexp', 'file_nd', 'file_sctr',
       'Name'],
      dtype='object')
21
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        18
           1       0.96      1.00      0.98        22

    accuracy                           0.97        40
   macro avg       0.98      0.97      0.97       

21
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       194
           1       1.00      1.00      1.00        20

    accuracy                           1.00       214
   macro avg       1.00      1.00      1.00       214
weighted avg       1.00      1.00      1.00       214

+++++++++++++++++   MarkdownView  +++++++++++++++++
Index(['commit_hash', 'file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
       'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
       'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
       'file_ns', 'file_exp', 'file_sexp', 'file_rexp', 'file_nd', 'file_sctr',
       'Name'],
      dtype='object')
21
              precision    recall  f1-score   support

           0       1.00      0.97      0.99        38
           1       0.92      1.00      0.96        12

    accuracy                           0.98        50
   macro avg       0.96      0.99      0.97      

21
              precision    recall  f1-score   support

           0       0.73      1.00      0.84        16
           1       1.00      0.33      0.50         9

    accuracy                           0.76        25
   macro avg       0.86      0.67      0.67        25
weighted avg       0.83      0.76      0.72        25

+++++++++++++++++   pandora  +++++++++++++++++
Index(['commit_hash', 'file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
       'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
       'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
       'file_ns', 'file_exp', 'file_sexp', 'file_rexp', 'file_nd', 'file_sctr',
       'Name'],
      dtype='object')
21
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       161
           1       0.97      1.00      0.99        34

    accuracy                           0.99       195
   macro avg       0.99      1.00      0.99       195


21
              precision    recall  f1-score   support

           0       0.96      0.93      0.95        29
           1       0.86      0.92      0.89        13

    accuracy                           0.93        42
   macro avg       0.91      0.93      0.92        42
weighted avg       0.93      0.93      0.93        42

+++++++++++++++++   SandVXposed  +++++++++++++++++
Index(['commit_hash', 'file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
       'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
       'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
       'file_ns', 'file_exp', 'file_sexp', 'file_rexp', 'file_nd', 'file_sctr',
       'Name'],
      dtype='object')
21
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       356
           1       1.00      1.00      1.00        20

    accuracy                           1.00       376
   macro avg       1.00      1.00      1.00       

...

Index(['commit_hash', 'file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
       'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
       'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
       'file_ns', 'file_exp', 'file_sexp', 'file_rexp', 'file_nd', 'file_sctr',
       'Name'],
      dtype='object')
21
              precision    recall  f1-score   support

           0       1.00      0.90      0.95        78
           1       0.93      1.00      0.97       112

    accuracy                           0.96       190
   macro avg       0.97      0.95      0.96       190
weighted avg       0.96      0.96      0.96       190

+++++++++++++++++   super-csv  +++++++++++++++++
Index(['commit_hash', 'file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
       'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
       'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
       'file_ns', 'file_exp', 'file_sexp', 

21
              precision    recall  f1-score   support

           0       0.99      0.95      0.97       601
           1       0.85      0.98      0.91       167

    accuracy                           0.96       768
   macro avg       0.92      0.97      0.94       768
weighted avg       0.96      0.96      0.96       768

+++++++++++++++++   jdeb  +++++++++++++++++
Index(['commit_hash', 'file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
       'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
       'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
       'file_ns', 'file_exp', 'file_sexp', 'file_rexp', 'file_nd', 'file_sctr',
       'Name'],
      dtype='object')
21
              precision    recall  f1-score   support

           0       0.92      0.92      0.92       183
           1       0.92      0.91      0.91       168

    accuracy                           0.92       351
   macro avg       0.92      0.92      0.92       351
wei

21
              precision    recall  f1-score   support

           0       0.99      0.90      0.94       552
           1       0.89      0.99      0.94       442

    accuracy                           0.94       994
   macro avg       0.94      0.95      0.94       994
weighted avg       0.95      0.94      0.94       994

+++++++++++++++++   restlet-framework-java  +++++++++++++++++
Index(['commit_hash', 'file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
       'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
       'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
       'file_ns', 'file_exp', 'file_sexp', 'file_rexp', 'file_nd', 'file_sctr',
       'Name'],
      dtype='object')
Shape of passed values is (4096, 45), indices imply (4096, 66)
+++++++++++++++++   addressbook-level4  +++++++++++++++++
Index(['commit_hash', 'file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
       'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm

21
              precision    recall  f1-score   support

           0       0.98      0.88      0.93       152
           1       0.89      0.98      0.93       144

    accuracy                           0.93       296
   macro avg       0.93      0.93      0.93       296
weighted avg       0.93      0.93      0.93       296

+++++++++++++++++   athenz  +++++++++++++++++
Index(['commit_hash', 'file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
       'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
       'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
       'file_ns', 'file_exp', 'file_sexp', 'file_rexp', 'file_nd', 'file_sctr',
       'Name'],
      dtype='object')
21
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       816
           1       0.92      0.97      0.94       292

    accuracy                           0.97      1108
   macro avg       0.95      0.97      0.96      1108
w

Index(['commit_hash', 'file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
       'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
       'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
       'file_ns', 'file_exp', 'file_sexp', 'file_rexp', 'file_nd', 'file_sctr',
       'Name'],
      dtype='object')
21
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      1538
           1       0.94      0.98      0.96       638

    accuracy                           0.97      2176
   macro avg       0.96      0.97      0.97      2176
weighted avg       0.97      0.97      0.97      2176

+++++++++++++++++   tx-lcn  +++++++++++++++++
Index(['commit_hash', 'file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
       'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
       'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
       'file_ns', 'file_exp', 'file_sexp', 'fi

In [74]:
# with open('results/Performance/commit_guru_file_specific/process_700_rf_5_fold_5_repeat.pkl', 'wb') as handle:
#     pickle.dump(final_result, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
for project in projects[150:]:
    try:
        if project == '.DS_Store':
            continue
        df1 = pd.read_csv('data/commit_guru_file_level/' + project + '_file.csv')
        df2 = pd.read_csv('data/commit_guru_file/' + project + '.csv')
        if df1.shape[0] == df2.shape[0]:
            print(project,df1.shape[0],df2.shape[0])
    except:
        continue

environment-dashboard 280 280
egads 373 373
dockerfile-maven 432 432
android-pdfview 477 477
rebound 512 512
logback-kafka-appender 334 334
Droppy 333 333
RxActivityResult 285 285
PatternLock 543 543
android-transcoder 229 229
libsignal-protocol-java 1027 1027
ESPlorer 6375 6375
CircularReveal 328 328
ABAGAIL 962 962
react-native-splash-screen 493 493
react-native-incall-manager 200 200
libsignal-service-java 1414 1414
MarkdownView 2323 2323
dcevm 870 870
wycheproof 1184 1184
FastDFS_Client 1152 1152
Telecine 742 742
Bolts-Android 514 514
gcm 1314 1314
android-app 1254 1254
Android-RateThisApp 466 466
progressbar 332 332
android-sliding-layer-lib 442 442
ASNE 2473 2473
TapTargetView 332 332
RxCache 1478 1478
EasyRefreshLayout 439 439
Recyclerview-Gallery 335 335
SandVXposed 1266 1266
clean-status-bar 597 597
DynamicGrid 179 179
bottomsheet 285 285
HomeMirror 497 497
emojicon 3157 3157
otto 548 548
ip2region 826 826
native-navigation 933 933
ExpandableTextView 571 571
Reservoir 252 252


swagger-maven-plugin 2374 2374
PretendYoureXyzzy 2136 2136
RxAndroidBle 3767 3767
BitcoinChecker 1421 1421
memcached-session-manager 2714 2714
JsonPath 3124 3124
RichTextFX 3162 3162
jetcache 5935 5935
crash 16153 16153
Grammar-Kit 5803 5803
macrobase 3866 3866
UltimateRecyclerView 2382 2382
openstack4j 8978 8978
artifactory-plugin 6006 6006
java-memcached-client 3629 3629
jline2 1887 1887
springside4 9895 9895
android-async-http 2081 2081
logbook 5222 5222
ActionBarSherlock 9581 9581
config 3166 3166
getdown 2311 2311
jzmq 1244 1244
spring-social-facebook 9170 9170
librec 7320 7320
multi-model-server 3829 3829
Android-Orma 4833 4833
mamute 15733 15733
Hystrix 4501 4501
giraph 13117 13117
astyanax 12924 12924
RSyntaxTextArea 6136 6136
malmo 4680 4680
stringtemplate4 3651 3651
chat-sdk-android 20956 20956
Duke 2866 2866
EhViewer 18231 18231
goclipse 21575 21575
Jest 4968 4968
flume 12369 12369
samourai-wallet-android 5905 5905
PageTurner 3559 3559
univocity-parsers 4406 4406
TrebleShot 

In [15]:
projects[]

['org.alloytools.alloy',
 'qpython',
 'friendlychat-android',
 'paho.mqtt.android',
 'paho.mqtt.java',
 'android-docs-samples',
 'Toasty',
 'android-mvvm-architecture',
 'recipes-rss',
 'HoloGraphLibrary',
 'HorizontalWheelView',
 'OpenMemories-Tweak',
 'HoldingButton',
 'ExpandableLayout',
 'RecyclerView-MultipleViewTypesAdapter',
 'UpdateApp',
 'activate-power-mode',
 'JDAddressSelector',
 'android-ago',
 'cordova-plugin-streaming-media',
 'birt',
 'arquillian-examples',
 'dex-method-counts',
 'google-authenticator-android',
 'android-samples',
 'PLDroidMediaStreaming',
 'hacktoberfest-2019',
 'egit-github',
 'Hello-World',
 'API-NG-sample-code',
 'openshift-quickstarts',
 'http-request',
 'NumberProgressBar',
 'eclipse.jdt.ls',
 'portfolio',
 'DataSphereStudio',
 'mapstruct-examples',
 'amazon-kinesis-connectors',
 'Android-ExpandIcon',
 'MyLittleCanvas',
 'Dachshund-Tab-Layout',
 'Pix-EzViewer',
 'FastScroll',
 'recyclerviewhelper',
 'MaterialFavoriteButton',
 'MaterialPreferenceLi

In [24]:
np.median(list(final_result['recall'].values()))

0.99

In [26]:
len(final_result['recall'].values())

654

In [43]:
len(errored)

161

In [44]:
proj_df_new = proj_df[proj_df.repo_name.isin(errored)]

In [45]:
proj_df_new.reset_index(drop=True,inplace=True)

In [47]:
proj_df_new.to_csv('projects_error.csv')

In [38]:
projects = proj_df_new.repo_name.tolist()

In [48]:
len(projects)

231

In [46]:
proj_df_new

,repo_name,repo_owner,git_url,api_base_url,source_type,access_token,lang,commits,defective
0,spring-xd-samples,spring-projects,https://github.com/spring-projects/spring-xd-s...,http://api.github.com,github_repo,NaN,Java,275,9
1,react-native-background-timer,ocetnik,https://github.com/ocetnik/react-native-backgr...,http://api.github.com,github_repo,NaN,Java,150,9
2,jgit,eclipse,https://github.com/eclipse/jgit,http://api.github.com,github_repo,NaN,Java,7746,11
3,android-app,eoecn,https://github.com/eoecn/android-app,http://api.github.com,github_repo,NaN,Java,132,11
4,BottomSheetBuilder,rubensousa,https://github.com/rubensousa/BottomSheetBuilder,http://api.github.com,github_repo,NaN,Java,138,11
...,...,...,...,...,...,...,...,...,...
156,XposedInstaller,rovo89,https://github.com/rovo89/XposedInstaller,http://api.github.com,github_repo,NaN,Java,1048,319
157,rtmp-rtsp-stream-client-java,pedroSG94,https://github.com/pedroSG94/rtmp-rtsp-stream-...,http://api.github.com,github_repo,NaN,Java,660,325
158,opsu,itdelatrisu,https://github.com/itdelatrisu/opsu,http://api.github.com,github_repo,6ff783294993b2f3c1d44506883d62f52ff91f80,Java,947,398
159,qbit,advantageous,https://github.com/advantageous/qbit,http://api.github.com,github_repo,6ff783294993b2f3c1d44506883d62f52ff91f80,Java,1717,492
